# TP4
Belén Götz

In [ ]:
from typing import Optional, Any, List

class Graph:
    """
    Graph class
    """
    def __init__(self):
        self._graph = {}

    def add_vertex(self, vertice: str, data: Optional[Any]=None) -> None:
        """
        Adds a vertice to the graph
        :param vertice: the vertice name
        :param data: data associated with the vertice
        """
        if vertice not in self._graph:
            self._graph[vertice] = {'data': data, 'vecinos': {}}

    def add_edge(self, vertice1: str, vertice2: str, data: Optional[Any]=None) -> None:
        """
        Adds an edge to the graph
        :param vertice1: vertice1 key
        :param vertice2: vertice2 key
        :param data: the data associated with the vertice
        """
        if not vertice1 in self._graph or not vertice2 in self._graph:
            raise ValueError("The vertexes do not exist")
        self._graph[vertice1]['vecinos'][vertice2] = data

    def get_neighbors(self, vertice) -> List[str]:
        """
        Get the list of vertice vecinos
        :param vertice: the vertice to query
        :return: the list of vecino vertexes
        """
        if vertice in self._graph:
            return list(self._graph[vertice]['vecinos'].keys())
        else:
            return []

    def get_vertex_data(self, vertice: str) -> Optional[Any]:
        """
        Gets  vertice associated data
        :param vertice: the vertice name
        :return: the vertice data
        """
        if self.vertex_exists(vertice):
            return self._graph[vertice]['data']
        else:
            return None

    def get_edge_data(self, vertice1: str, vertice2: str) -> Optional[Any]:
        """
        Gets the vertexes edge data
        :param vertice1: the vertice1 name
        :param vertice2: the vertice2 name
        :return: vertexes edge data
        """
        if self.edge_exists(vertice1, vertice2):
            return self._graph[vertice1]['vecinos'][vertice2]
        raise ValueError("The edge does not exist")

    def print_graph(self) -> None:
        """
        Prints the graph
        """
        for vertice, data in self._graph.items():
            print("vertice:", vertice)
            print("Data:", data['data'])
            print("vecinos:", data['vecinos'])
            print("")

    def vertex_exists(self, vertice: str) -> bool:
        """
        If contains a vertice
        :param vertice: the vertice name
        :return: boolean
        """
        return vertice in self._graph

    def edge_exists(self, vertice1: str, vertice2: str) -> bool:
        """
        If contains an edge
        :param vertice1: the vertice1 name
        :param vertice2: the vertice2 name
        :return: boolean
        """
        return vertice1 in self._graph and vertice2 in self._graph[vertice1]['vecinos']
    
    #agregado x mi
    def add_edge_reciprocal(self, vertice1: str, vertice2: str, data: Optional[Any]=None) -> None:
        """
        Adds an edge to the graph
        :param vertice1: vertice1 key
        :param vertice2: vertice2 key
        :param data: the data associated with the vertice
        THAT but BIDIRECCIONAL <->
        """
        if not vertice1 in self._graph or not vertice2 in self._graph:
            raise ValueError("The vertexes do not exist")
        self._graph[vertice1]['vecinos'][vertice2] = data
        self._graph[vertice2]['vecinos'][vertice1] = data

    def get_predecessors(self, vertice: str) -> List[str]:
        """
        Gets the list of predecessors (nodes pointing to the given vertice).
        :param vertice: the vertice to query
        :return: the list of predecessor vertexes
        """
        if not self.vertex_exists(vertice):
            raise ValueError(f"vertice '{vertice}' does not exist in the graph.")

        predecessors = []
        for v, data in self._graph.items():
            if vertice in data['vecinos']:
                predecessors.append(v)
        return predecessors
    
    def get_vertexes(self) -> List[str]:
        """
        Devuelve una lista con todos los vértices del grafo.
        :return: Lista de nombres de vértices.
        """
        return list(self._graph.keys())
    
    def get_neighbors_with_weight(self, node: str) -> List[tuple]:
        """
        Devuelve los vecinos de un nodo y sus respectivos pesos.
        :param node: Nodo del cual queremos obtener los vecinos
        :return: Lista de tuplas (vecino, peso)
        """
        if node in self._graph:
            return [(vecino, self._graph[node]['vecinos'][vecino]) for vecino in self._graph[node]['vecinos']]
        else:
            return []

    

In [2]:
graph = Graph()

#### CARGAR GRAFO:
###### 2m 35s/3m:

In [3]:
from tqdm import tqdm

index_title_dict = {}

with open("vertexes.txt", "r") as file:
    for line in file:
        index = int(line.split(" ")[0])
        title = " ".join(line[:-1].split(" ")[1:])
        index_title_dict[index] = title
        graph.add_vertex(title)

with open("edges.txt", "r") as file:
    for line in tqdm(file, desc="Loading edges", total=101409330):
        art1 = int(line[:-1].split(" ")[0])
        art2 = int(line[:-1].split(" ")[1])
        q = int(line[:-1].split(" ")[2])
        if art1 in index_title_dict and art2 in index_title_dict:
            graph.add_edge(index_title_dict[art1], index_title_dict[art2], q)

del index_title_dict

Loading edges: 100%|█████████▉| 101409095/101409330 [02:29<00:00, 678042.50it/s]


#### DFS código

In [4]:
from collections import deque

def dfs(graph: Graph, vertice: str, visitados: set, pbar: tqdm):
    stack = [vertice]
    while stack:
        current = stack.pop()
        if current not in visitados:
            visitados.add(current)
            stack.extend(
                vecino for vecino in graph.get_neighbors(current) if vecino not in visitados
            )
            pbar.update(1)        

# 1) 
¿Es el grafo débilmente conexo? Si no es así, ¿cuántas componentes conexas tiene? 

Se vuelve a cargar el grafo como "NO dirigido" (técnicamente con todas sus artistas bidireccionales) y se cuentan los componentes.

###### 5m 24s:

In [15]:
def convertir_a_NO_dirigido(graph: Graph) -> Graph:
    grafo_NOdirigido = Graph()

    for vertice in tqdm(graph.get_vertexes(), desc="Agregando vértices al grafo no dirigido"):
        grafo_NOdirigido.add_vertex(vertice, graph.get_vertex_data(vertice))

    for vertice in tqdm(graph.get_vertexes(), desc="Agregando aristas al grafo no dirigido"):
        for vecino, peso in graph.get_neighbors_with_weight(vertice):
            if not grafo_NOdirigido.edge_exists(vertice, vecino):
                grafo_NOdirigido.add_edge_reciprocal(vertice, vecino, peso)
                
    return grafo_NOdirigido

def contar_componentes(graph: Graph) -> int:
    vertexes_todos = graph.get_vertexes()
    visitados = set()
    componentes = 0

    with tqdm(total=len(vertexes_todos), desc="Contando componentes") as pbar:
        for vertice in vertexes_todos:
            if vertice not in visitados:
                componentes += 1
                dfs(graph, vertice, visitados, pbar)

    return componentes

def analizar_conectividad(graph: Graph):

    print("Convirtiendo grafo a NO dirigido...")
    grafo_NOdirigido = convertir_a_NO_dirigido(graph)
    del graph #para que no explote mi compu

    print("Cargando conectividad...")
    num_componentes = contar_componentes(grafo_NOdirigido)

    if num_componentes == 1:
        print("El grafo es débilmente conexo :)")
    else:
        print(f"El grafo NO es débilmente conexo. Número de componentes conexas: {num_componentes:_}")

analizar_conectividad(graph)

Convirtiendo grafo a NO dirigido...


Agregando aristas al grafo no dirigido: 100%|██████████| 3958452/3958452 [03:02<00:00, 21706.50it/s]


Cargando conectividad...


Contando componentes: 100%|██████████| 3958452/3958452 [00:53<00:00, 74173.27it/s] 


El grafo NO es débilmente conexo. Número de componentes conexas: 32_665


# 2) 
Wikipedia race: Existe un juego que consiste en dadas dos páginas de wikipedia los jugadores compiten por llegar desde la primera página hasta la segunda en la menor cantidad de o bien tiempo o bien links apretados. Por ejemplo, si fuera por cantidad de links, para ir de la página de “Samsung Galaxy J7 Prime” a la de “Guillermo Francella” un camino más corto es:

    1. Samsung Galaxy J7 Prime
    2. Samsung Galaxy Z Flip
    3. 92nd Academy Awards
    4. Antonio Banderas
    5. Guillermo Francella


Realice una función que dados dos títulos de Wikipedia pueda buscar el camino mínimo entre ambos.

In [6]:
from typing import Tuple, List
from collections import deque

def bfs_camino_corto(graph, start: str, end: str):
    if start not in graph._graph or end not in graph._graph:
        return [], float('inf')

    queue = deque([(start, [start])])
    visitados = set()

    while queue:
        current_node, camino = queue.popleft()

        if current_node == end:
            return camino, len(camino) - 1

        if current_node not in visitados:
            visitados.add(current_node)
            for vecino in graph.get_neighbors(current_node):
                if vecino not in visitados:
                    queue.append((vecino, camino + [vecino]))

    return [], float('inf')


In [7]:
start = "Samsung Galaxy J7 Prime"
end = "Guillermo Francella"

camino_minimo, peso_total = bfs_camino_corto(graph, start, end)

if camino_minimo:
    print("Camino más corto:")
    print(" -> ".join(camino_minimo))
    print(f"Peso total: {peso_total}")
else:
    print("No hay un camino entre los nodos especificados.")


Camino más corto:
Samsung Galaxy J7 Prime -> Android -> GNU General Public License -> 25 de febrero -> 14 de febrero -> Guillermo Francella
Peso total: 5


# 3) 
Queremos puntuar también a los competidores por originalidad en sus caminos. Solo los competidores que consigan los caminos con la menor cantidad de saltos pueden puntuar, pero no todos los caminos son igual de originales. Decimos que un camino es “poco original” si su promedio de pesos de aristas es alto (son los links que se repetían más veces) y es “muy original” si su promedio de aristas es el más bajo. Dadas dos aristas devuelva de todos los caminos mínimos el más y el menos original. Muestre un ejemplo.

Para puntuar los caminos por originalidad primero encuentro la lista de todos los caminos posibles para una dada profundidad (mínima).
Con BFS, dado el nodo inicial, ramificamos hasta encontrar un 'hit' al nodo end.
Una vez encontrada 'hit' continuamos pero a la misma profundida.

###### 1m 36s:

In [18]:
def buscar_todos_posibles(start, end):
    if not graph.vertex_exists(start) or not graph.vertex_exists(end):
        raise ValueError("Los nodos de inicio o final no son validos.")
    queue = deque([[start]])
    camino_minimo = []
    visited_at_level = {start: 0}
    while queue:
        path = queue.popleft()
        current_node = path[-1]
        if current_node != end:
            for vecino in graph.get_neighbors(current_node):
                if vecino not in visited_at_level or visited_at_level[vecino] == len(path):
                    if vecino not in visited_at_level:
                        visited_at_level[vecino] = len(path)
                    queue.append(path + [vecino])
        else:      
            camino_minimo.append(path)
            continue
    return camino_minimo

posibles_caminos = buscar_todos_posibles('Samsung Galaxy J7 Prime', 'Guillermo Francella')
print(f"CAMINOS: cantidad {len(posibles_caminos)}, caminos: {posibles_caminos}")

CAMINOS: cantidad 234, caminos: [['Samsung Galaxy J7 Prime', 'Android', 'GNU General Public License', '25 de febrero', '14 de febrero', 'Guillermo Francella'], ['Samsung Galaxy J7 Prime', 'Android', 'GNU General Public License', '25 de febrero', '1955', 'Guillermo Francella'], ['Samsung Galaxy J7 Prime', 'Android', 'GNU General Public License', '1989', '14 de febrero', 'Guillermo Francella'], ['Samsung Galaxy J7 Prime', 'Android', 'Estados Unidos', 'Primer contacto', 'Enrique el Navegante', 'Guillermo Francella'], ['Samsung Galaxy J7 Prime', 'Android', 'Estados Unidos', 'Orson Welles', 'Plano secuencia', 'Guillermo Francella'], ['Samsung Galaxy J7 Prime', 'Android', 'Estados Unidos', 'Francis Ford Coppola', 'Norma Pons', 'Guillermo Francella'], ['Samsung Galaxy J7 Prime', 'Android', 'Estados Unidos', 'Emily Dickinson', 'Silvina Ocampo', 'Guillermo Francella'], ['Samsung Galaxy J7 Prime', 'Android', 'Aptoide', 'Portugal', 'Enrique el Navegante', 'Guillermo Francella'], ['Samsung Galaxy 

Encuentro el minimo y máximo

In [19]:
def find_minmax_weights(paths):
    min_peso_promedio = float('inf')
    camino_min = []
    max_peso_promedio = float('-inf')
    camino_max = []
    for camino in paths:
        peso_total = 0
        for i in range(len(camino) - 1):
            peso_total += graph.get_edge_data(camino[i], camino[i + 1])
        peso_promedio = peso_total / (len(camino) - 1)
        if peso_promedio < min_peso_promedio:
            min_peso_promedio = peso_promedio
            camino_min = camino
        if peso_promedio > max_peso_promedio:
            max_peso_promedio = peso_promedio
            camino_max = camino
    return ((camino_min, min_peso_promedio), (camino_max, max_peso_promedio))

In [20]:
minmax_paths = find_minmax_weights(posibles_caminos)

print(f"Camino más original: {minmax_paths[0][0]} con peso promedio {minmax_paths[0][1]}")
print(f"Camino menos original: {minmax_paths[1][0]} con peso promedio {minmax_paths[1][1]}")

Camino más original: ['Samsung Galaxy J7 Prime', 'Android', 'GNU General Public License', '25 de febrero', '14 de febrero', 'Guillermo Francella'] con peso promedio 1.0
Camino menos original: ['Samsung Galaxy J7 Prime', '4G', 'Uruguay', 'Argentina', 'El secreto de sus ojos', 'Guillermo Francella'] con peso promedio 6.4


# 4) 
¿Cuál es el camino mínimo más largo de todo el grafo (sin tener ciclos en cuenta)? Muestre todos los vértices por los que pasa. En caso de no poder calcularlo de forma exacta haga un algoritmo que pueda encontrar el más largo posible.

NO CORRE EL EXACTO, entonces trabajamos con una APROXIMACIÓN (estimación).

Se estima el diámetro mediante varios intentos (bfs modificado) devolviendo el camino más largo obtenido.

In [11]:
import random
import random

def bfs_longest_path(graph: Graph, start: str):
    queue = deque([(start, [start])])
    visited = set([start])
    longest_path = []

    while queue:
        current_vertex, path = queue.popleft()
        if len(path) > len(longest_path):
            longest_path = path

        for neighbor in graph.get_neighbors(current_vertex):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))

    return longest_path


def estimar_diametro(grafo, intentos):
    max = []
    for i in range(intentos):    
        nodos = list(grafo._graph.keys())
        nodo_inicial = random.choice(nodos)
        while (len(grafo.get_neighbors(nodo_inicial)) < 100):
            nodo_inicial = random.choice(nodos)

        current = bfs_longest_path(graph, nodo_inicial)
        if len(current) > len(max):
            max = current
    
    return max

###### 30s:

In [16]:
camino = estimar_diametro(graph, 10)

print(f"Longitud del camino más largo encontrado: {len(camino)}")

if camino:
    print("Camino completo")
    print(" -> ".join(camino))
    print("\nPrimeros 5 nodos del camino:")
    print(" -> ".join(camino[:5]))
    
    print("\nÚltimos 5 nodos del camino:")
    print(" -> ".join(camino[-5:]))
else:
    print("No se encontró un camino.")
    


Longitud del camino más largo encontrado: 37
Camino completo
Idiomas aislados -> Francia -> Suiza -> Roger Federer -> Francisco Clavet -> Masters de Cincinnati 2000 -> Masters de Cincinnati 1999 -> Masters de Cincinnati 1998 -> Masters de Cincinnati 1997 -> Masters de Cincinnati 1996 -> Masters de Cincinnati 1995 -> Masters de Cincinnati 1994 -> Masters de Cincinnati 1993 -> Masters de Cincinnati 1992 -> Masters de Cincinnati 1991 -> Masters de Cincinnati 1990 -> Masters de Cincinnati 1989 -> Masters de Cincinnati 1988 -> Masters de Cincinnati 1987 -> Masters de Cincinnati 1986 -> Masters de Cincinnati 1985 -> Masters de Cincinnati 1984 -> Masters de Cincinnati 1983 -> Masters de Cincinnati 1982 -> Masters de Cincinnati 1981 -> Masters de Cincinnati 1980 -> Masters de Cincinnati 1979 -> Masters de Cincinnati 1978 -> Masters de Cincinnati 1977 -> Masters de Cincinnati 1976 -> Masters de Cincinnati 1975 -> Masters de Cincinnati 1974 -> Masters de Cincinnati 1973 -> Masters de Cincinnati 

# 5) 
¿Cuántos artículos hay para los cuales no se llegue desde ningún otro artículo? ¿Cuántos artículos que no lleven a ningún otro hay? Muestre 5 de cada uno.

Se buscan los nodos con entradas y salidas, y los que no tienen entrada o no tienen salida son los que no están ahí.

In [13]:
def encontrar_articulos_sin_conexiones(grafo):
    nodos_con_entrada = set()
    nodos_con_salida = set()
    
    for nodo in grafo._graph:
        vecinos = grafo.get_neighbors(nodo)
        if vecinos:
            nodos_con_salida.add(nodo)
            for vecino in vecinos:
                nodos_con_entrada.add(vecino)
    
    todos_nodos = set(grafo._graph.keys())
    sin_entrada = todos_nodos - nodos_con_entrada
    sin_salida = todos_nodos - nodos_con_salida
    
    return list(sin_entrada), list(sin_salida)

sin_entrada, sin_salida = encontrar_articulos_sin_conexiones(graph)

print(f"Cantidad de artículos sin enlaces entrantes: {len(sin_entrada)}")
print(f"Primeros 5 artículos sin enlaces entrantes:")
for articulo in sin_entrada[:5]:
    print(f"- {articulo}")
print(f"\nCantidad de artículos sin enlaces salientes: {len(sin_salida)}")
print(f"Primeros 5 artículos sin enlaces salientes:")
for articulo in sin_salida[:5]:
    print(f"- {articulo}")

Cantidad de artículos sin enlaces entrantes: 2958256
Primeros 5 artículos sin enlaces entrantes:
- Cereus poselgerianus
- Escudo de armas de Túnez
- Rab (empresa)
- Ermita de Nuestra Señora de Mismanos (Tudela)
- Macrófitas

Cantidad de artículos sin enlaces salientes: 32793
Primeros 5 artículos sin enlaces salientes:
- Alberto Lara
- Taishō
- Orland
- Mendez
- Chapeau (desambiguación)


# 6) 
¿Cuántos artículos con links correspondidos hay (un link de A a B y de B a A)? Muestra 5 pares.

Se buscan directamente.

###### 1m

In [14]:
def encontrar_links_correspondidos(grafo):
    pares_correspondidos = []
    nodos_visitados = set()
    
    for nodo in tqdm(grafo._graph, desc="Explorando pares correspondidos..."):
        if nodo in nodos_visitados:
            continue

        for vecino in grafo.get_neighbors(nodo):
            if grafo.edge_exists(vecino, nodo):
                pares_correspondidos.append((nodo, vecino))
                nodos_visitados.add(nodo)
                nodos_visitados.add(vecino)

    return nodos_visitados, pares_correspondidos


nodos_visitados, pares_correspondidos = encontrar_links_correspondidos(graph)
print(f"Cantidad de artículos con links correspondidos: {len(nodos_visitados)}")
print(f"Primeros 5 pares de artículos con links correspondidos:")
for nodo1, nodo2 in pares_correspondidos[:5]:
    print(f"- {nodo1} <-> {nodo2}")

Explorando pares correspondidos...: 100%|██████████| 3958452/3958452 [00:54<00:00, 72067.78it/s]

Cantidad de artículos con links correspondidos: 728779
Primeros 5 pares de artículos con links correspondidos:
- Pictilabrus viridis <-> Pictilabrus
- Stenoma cremastis <-> Stenoma
- Excoecaria <-> Hippomaneae
- Excoecaria <-> Hippomaninae
- Melissa De Sousa <-> 30 Years to Life
